In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from src.features.seeds import SeedFeatures
from src.utils import load_data_template

In [3]:
data = load_data_template(season=False)
data.dropna(subset=['a_win'], inplace=True)
seed_feat = SeedFeatures()
data = seed_feat.per_team_wrapper(data, seed_feat.team_seeds, per_game=False, per_day=False, combine='subtract')
data.dropna(inplace=True)
print(data.shape)
data.sample()

(2117, 8)


,Season,team_a,team_b,in_target,game_set,a_win,DayNum,seed_team_combined
10461,2006,1206,1277,False,0,True,137,5


In [4]:
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss


X = data.drop(['Season', 'team_a', 'team_b', 'a_win', 'in_target', 'DayNum'], axis=1)
y = data[['a_win']].astype(int)

kf = KFold(n_splits=5, shuffle=True)
metrics = []
for tr_i, t_i in kf.split(X):
    X_tr, y_tr = X.iloc[tr_i], y.iloc[tr_i]
    X_t, y_t = X.iloc[t_i], y.iloc[t_i]
    xgb = XGBClassifier()
    xgb.fit(X_tr.values, y_tr.values.reshape(-1))
    preds = xgb.predict_proba(X_t.values)
    metric = log_loss(y_t.values, preds)
    print(metric)
    metrics.append(metric)
    
print('Metric Mean: {:.2f} ({:.2f})'.format(np.mean(metrics), np.std(metrics)))

0.5603092693600733
0.5379983140375325
0.5836619116942757
0.5232136061648457
0.5420952387700126
Metric Mean: 0.55 (0.02)
